In [1]:
import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import sys
import time

from torch import nn

import asyncio ,nest_asyncio
from python_tools.discord_bot import async_discord_bot_notifier ,discord_bot_notifier
nest_asyncio.apply()


### control line

In [2]:
# csv_file = '../data/img_conv2.csv'
random_seed = 42
# dataset_path = "../data/HAM10000/images/"
# groundtruth_file = '../data/HAM10000/GroundTruth.csv'
# feature_vector_file_path = '../data/HAM10000/img_feature_no_masked.csv'

device = "cuda" if torch.cuda.is_available() else "cpu"

model_select_signal = 'resnet18'
# model_select_signal = 'vgg16'

data_select_signal = 'skin'
# data_select_signal = 'chest CT'

# select image type is RGB or not
isRGB = True
# isRGB = False

# enable img feature vector as mutimodal
enable_muti_modal_signal = True
# enable_muti_modal_signal = False

## dataset prepocessing

In [3]:
from helperFunction.CustomImageDataset import CustomImageDataset
import os
import random
import torchvision
import json

# 數據轉換
transform_std = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform224 = torchvision.transforms.Compose([
    transforms.Resize(size=(224,224)),
    torchvision.transforms.ToTensor()
    # 其他轉換
])

transform = transform_std


# all_files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]

# random.seed(42)

# num_total_samples = len(all_files)
# split_ratio = 0.8  # 80% 的数据用于训练，20% 用于测试

# # 随机打乱数据集
# random.shuffle(all_files)

# # 计算分割点
# split_idx = int(num_total_samples * split_ratio)

# # 分割数据集
# train_files = all_files[:split_idx]
# test_files = all_files[split_idx:]

# 載入train & test file list
# with open('test_files_list.json', 'r') as f:
#      test_files = json.load(f)
# with open('train_files_list.json', 'r') as f:
#      train_files = json.load(f)


# # 加載數據
# train_dataset = CustomImageDataset(img_dir=dataset_path,file_to_label_dict={file: filename_to_label_dict[file] for file in train_files}, transform=transform)
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)

# test_dataset = CustomImageDataset(img_dir=dataset_path,file_to_label_dict={file: filename_to_label_dict[file] for file in test_files}, transform=transform)
# test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
# @ transfrom test unit

# import unittest
# import numpy as np
# import torch
# from torchvision import transforms
# from PIL import Image

# class TestTransforms224Gray32bit(unittest.TestCase):
#     def setUp(self, img_path=None):
#         # 创建一个纯白色的测试图像，尺寸为 300x300
#         if img_path :
#             self.image = Image.open(img_path).convert('L')
#         else :
#             self.image = Image.new('L', (300, 300), color='white')

#     def test_transform_flow(self):
#         transform224_gray_32bit = transforms.Compose([
#             transforms.Lambda(lambda img: np.array(img).astype(np.float32) / 255.0),
#             transforms.Lambda(lambda x: torch.from_numpy(x)),
#             transforms.Resize(256),
#             transforms.CenterCrop(224),
#             transforms.Normalize(mean=[0.485], std=[0.229]),
#         ])
        
#         # 应用转换流程
#         transformed_img = transform224_gray_32bit(self.image)
        
#         # 检查转换后的类型和形状
#         self.assertTrue(isinstance(transformed_img, torch.Tensor), "Output should be a Torch Tensor")
#         self.assertEqual(transformed_img.dtype, torch.float32, "Output tensor should have dtype float32")
#         self.assertEqual(transformed_img.size(), (1, 224, 224), "Output tensor should have shape (1, 224, 224)")
        
#         # 检查值的范围是否合理（因为原图是纯白的，所以归一化后应该有一个固定的范围）
#         expected_value = (1 - 0.485) / 0.229
#         self.assertTrue(torch.allclose(transformed_img.mean(), torch.tensor(expected_value), atol=1e-5),
#                         "Normalized values do not match expected range")

# test = TestTransforms224Gray32bit()
# test.setUp('../data/chestCTData/images/train/adenocarcinoma/000000 (6).png')

# test.test_transform_flow()

In [5]:
sys.path.append(os.path.abspath('..'))


from data.HAM10000.ham10000Dataloader import HAM10000DataProcessor
from data.chestCTData.chestCTDataloader import ChestCTDataProcessor

if data_select_signal == 'skin':
    dataContainer = HAM10000DataProcessor(transform=transform_std)
elif data_select_signal == 'chest CT':
    dataContainer = ChestCTDataProcessor(transform=transform_std)
else:
    raise ValueError('需要指定dataset類別')

train_dataloader , test_dataloader = dataContainer.returnDataloaders()
train_files , test_files = dataContainer.returnDatasetFilenames()
feature_vector_file_path = dataContainer.returnFeatureVectorFilename()

num_classes = dataContainer.getNumClasses()



In [6]:
print(type(transform224))

# print(train_files[0:5])

<class 'torchvision.transforms.transforms.Compose'>


### load color feature data base on dataloader filename idx

In [7]:
# from helperFunction.XgbHelperFunction import csvkeylistToData


# train_feature_vectors = csvkeylistToData(feature_vector_file_path, train_files)
# test_feature_vectors = csvkeylistToData(feature_vector_file_path, test_files)

### cal img feature vectors

In [8]:
from helperFunction.helperFunctions import dataloaderToFeatureData , calImgFeatureVector
train_img_feature_vector = []
test_img_feature_vector = []

def print_list_dimensions(lst):
    dimensions = []
    while isinstance(lst, list):
        dimensions.append(len(lst))
        lst = lst[0] if len(lst) > 0 else []
    print("Dimensions:", " x ".join(map(str, dimensions)))

    return


if enable_muti_modal_signal:

    for idx , (batch_data, label) in enumerate(train_dataloader):
        for img in batch_data:
            train_img_feature_vector.append(calImgFeatureVector(img, isRGB=isRGB))
    
    # print_list_dimensions(train_img_feature_vector)

    for idx , (batch_data, label) in enumerate(test_dataloader):
        for img in batch_data:
            test_img_feature_vector.append(calImgFeatureVector(img, isRGB=isRGB))
    
    # print_list_dimensions(test_feature_vector)
        
    # for idx , (data, label) in enumerate(test_dataloader):
    #     train_img_feature_vector.append( calImgFeatureVector(data, isRGB=isRGB) )

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\skimage\feature\texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


In [9]:
print(test_img_feature_vector[0])

[0.01753672 0.12859136 0.14988535 0.12289615 0.27123658 0.19446339
 0.26082893 0.05883291 0.06812022 0.08902663 0.14245855 0.17151626
 0.14090402 0.09044165 0.06666534 0.0598294  0.11220504 0.30736113
 0.21707693 0.88701373 0.26760662 0.         0.         0.
 0.         0.77097738 0.62835687 0.1037376  0.         0.
 0.         0.         0.         1.         0.         0.
 0.         0.         0.         0.         0.        ]


### define XGB eval recorder function

In [47]:
# define XGB training function
from helperFunction.helperFunctions import dataloaderToFeatureData , calImgFeatureVector
from helperFunction.XgbHelperFunction import  train_predict, calBestIterOfXGB
import xgboost as xgb


def recordXGBoutput(model:nn.Sequential, train_dataloader, test_dataloader, enable_muti_module:bool=False, isRGB:bool=False):
  print("cal CNN model output...")
  test_features , test_labels = dataloaderToFeatureData(model, test_dataloader,device)
  
  start_time = time.time()
  train_features , train_labels = dataloaderToFeatureData(model, train_dataloader, device)
  end_time = time.time()
  CNN_eval_time = (end_time - start_time)/len(train_features)

  # muti modal selection
  if enable_muti_module:

    train_muti_modal_vectors = []
    for idx , data in enumerate(train_features):
      train_muti_modal_vectors.append(np.concatenate((train_features[idx] , train_img_feature_vector[idx])))
    train_features = np.array(train_muti_modal_vectors)

    test_muti_modal_vectors = []
    for idx , data in enumerate(test_features):
      test_muti_modal_vectors.append(np.concatenate((test_features[idx] , test_img_feature_vector[idx])))
    test_features = np.array(test_muti_modal_vectors)



  print("feature size is : {}".format(test_features.shape))
  # print_list_dimensions(test_features)
  
  print(feature for feature in test_features if len(feature) == 0)
  nan_indices = np.where(np.isnan(test_features))
  print(f'Indices of NaN values: {list(zip(nan_indices[0], nan_indices[1]))}')

  
  xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=num_classes)

  iter ,f1 ,acc, XGB_eval_time = calBestIterOfXGB(train_features, train_labels, test_features, test_labels, enable_f1_metric=True)

  # f1, acc = train_predict(xgb_model, train_features, train_labels,  test_features, test_labels)

  print("======eval finish!=========")

  return f1, acc, iter, CNN_eval_time, XGB_eval_time
  


### define ML eval recorder function

In [45]:
def recordMLoutput(ML_model, model:nn.Sequential, train_dataloader, test_dataloader, enable_muti_module=False):
  print("cal CNN model output...")
  test_features , test_labels = dataloaderToFeatureData(model, test_dataloader,device)
  str_time = time.time()
  # train_features , train_labels = dataloaderToFeatureData(model, train_dataloader, device)


  if enable_muti_module:
    # train_muti_modal_vectors = []
    # for idx , data in enumerate(train_features):
    #   train_muti_modal_vectors.append(np.concatenate((train_features[idx] , train_img_feature_vector[idx])))
    # train_features = np.array(train_muti_modal_vectors)

    test_muti_modal_vectors = []
    for idx , data in enumerate(test_features):
      test_muti_modal_vectors.append(np.concatenate((test_features[idx] , test_img_feature_vector[idx])))
    test_features = np.array(test_muti_modal_vectors)

  # print("????")
  # print(test_img_feature_vector[0])
  print(feature for feature in test_features if len(feature) == 0)
  nan_indices = np.where(np.isnan(test_features))
  print(f'Indices of NaN values: {list(zip(nan_indices[0], nan_indices[1]))}')
  
  # print(len(test_features[0]))
  f1, acc = train_predict(ML_model, train_features, train_labels,  test_features, test_labels)

  print("======eval finish!=========")

  return f1, acc 

## init nn module

### 初始化模型

In [12]:
from torchinfo import summary

if data_select_signal == 'skin':
    model_folder_path = "../model/HAM10000"
elif data_select_signal == 'chest CT':
    model_folder_path = "../model/CT chest"



# resnet101 = models.resnet101(pretrained=True)
# #  ===================================
# # 加載預訓練的ResNet模型
# resnet18 = models.resnet18(pretrained=True)
# resnet18 = torch.nn.Sequential(*(list(resnet18.children())[:-1]))  # 移除最後的全連接層

#  ===================================
# 加載訓練好的ResNet模型
resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, num_classes)
resnet18.load_state_dict(torch.load(model_folder_path + "/best_model_pretrain_Resnet18.pth"))
# resnet18_7  = torch.nn.Sequential(*(list(resnet18_7.children())[:-1]))  # 移除最後的全連接層

# #  =========================
# resnet50 = models.resnet50(pretrained=True)
# num_ftrs = resnet50.fc.in_features
# resnet50.fc = nn.Linear(num_ftrs, num_classes)
# resnet50.load_state_dict(torch.load(model_folder_path + "/best_model_pretrain_Resnet50_7.pth"))

# #  ===================================
# # 加載預訓練的VGG模型
# vgg16 = models.vgg16(pretrained=True)
# vgg16.classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-1]) # 移除最後的全連接層

#  ===================================
# 載入訓練好的vgg
vgg16 = models.vgg16(pretrained=True)
classifier = list(vgg16.classifier.children())[:-1]

# 移除原始模型的最后一个全连接层
# 并添加一个新的全连接层，输出特征数为 輸出的種類數
classifier.append(torch.nn.Linear(4096, num_classes))

# 替换原始模型的分类器
vgg16.classifier = torch.nn.Sequential(*classifier)

vgg16.load_state_dict(torch.load(model_folder_path + "/best_model_pretrain_VGG16.pth"))

# 使用nn.Sequential的方式取代torch.flatten的功能
new_classfier = nn.Sequential(
    nn.Flatten(),
    vgg16.classifier,
)

vgg16.classifier = new_classfier

summary(vgg16, [1,3,224,224])
# vgg16.classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-1]) # 移除最後的全連接層


c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`.

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 7]                    --
├─Sequential: 1-1                        [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [1, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [1, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          29

In [13]:
# 決定使用的模型
model_0 = None

if model_select_signal == 'resnet18':
    model_0 = resnet18
elif model_select_signal == 'vgg16':
    model_0 = vgg16

model_0 = model_0.to(device)

summary(model_0, input_size=[1,3,224,224])

# get var name
model_0_name = [name for name, val in globals().items() if val == model_0][0]

In [14]:
test_features , test_labels = dataloaderToFeatureData(model_0, test_dataloader,device)
train_features , train_labels = dataloaderToFeatureData(model_0, train_dataloader, device)


# for idx , data in enumerate(train_features):
#     train_features[idx] = np.concatenate(train_features[idx] , train_img_feature_vector[idx])



feature size is :2003
feature size is :8012


In [15]:
# @ function test unit : dataloaderToFeatureData w/ img feature enhence
# train_features , train_labels = dataloaderToFeatureData(model_0, train_dataloader, device)

# print(len(train_features))
# print(type(train_features))
# print(train_features[0])

In [16]:
# @ unit test : find specific layers

# import torchvision.models as models

# # 加载预训练的ResNet18模型
# model_0 = models.resnet18(pretrained=True)

# # 初始化层计数器
# total_layers = 0

# # 遍历模型的所有子模块和层
# for name, layer in model_0.named_modules():
#     # 打印每一层的名称和它的具体类型，这一步是可选的，但对理解模型结构很有帮助
#     # print(name, layer.__class__.__name__)

#     # 对所有层进行计数（包括卷积层、全连接层等）
#     # 如果只想计算特定类型的层（如卷积层Conv2d），则需要添加判断条件
#     total_layers += 1

# # 打印总层数
# print(f'Total number of layers: {total_layers}')

# # 示例：仅计算Conv2d层的数量
# conv_layers = 0
# for name, layer in model_0.named_modules():
#     if isinstance(layer, torch.nn.Sequential):
#         print(name, layer.__class__.__name__)
#         conv_layers += 1

# print(f'Total number of Sequential layers: {conv_layers}')

### 建立輸出為不同隱藏層的model

In [17]:
from helperFunction.helperFunctions import createDetailLayerVersions 
# detail version

list_of_models = createDetailLayerVersions(model_0)

# block level version

# list_of_models = []

# layer = 10
# list_of_models.append((model_0 , "layer:"+str(layer)))
# model = model_0

# while layer > 0:
#     model =  torch.nn.Sequential(*(list(model.children())[:-1])) 
#     layer -= 1
#     list_of_models.append((model , "layer:"+str(layer)))

# print(list_of_models)

# len(list(model.children()))

總層數為: 68層


In [18]:
# @ unit test : eval ability of model, in list of models
# summary(model_0, input_size=[1,3,224,224])
# summary(list_of_models[67][0] , input_size=[1,3,224,224])
# summary(list_of_models[66][0] , input_size=[1,3,224,224])
# summary(list_of_models[0][0] , input_size=[1,3,224,224])

In [19]:
# @ unit test : dataloaerToFeatureData()

# # model.forward( {data in datalaoder} ) -> (features , label)
# from helperFunction.helperFunctions import dataloaderToFeatureData

# model_0 = torch.nn.Sequential(*(list(model_0.children())[:-1])) 

# test_features , test_labels = dataloaderToFeatureData(model_0, test_dataloader,device)
# train_features , train_labels = dataloaderToFeatureData(model_0, train_dataloader, device)



In [20]:
# @ model test : 確認載入的model性能與原本相符

from helperFunction.TrainHelper import TrainingHelper

train_helper = TrainingHelper(model_0,
                              train_dataloader=train_dataloader,
                              test_dataloader=test_dataloader,
                              loss_fn=nn.CrossEntropyLoss(),
                              optimizer=torch.optim.SGD(model_0.parameters(), lr = 0.001),
                              device=device)

print(train_helper.test_step())


c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(0.5025377235715351, 0.8467000835421888, 0.8393717787047136)


In [21]:
# from helperFunction.helperFunctions import calDetailModelLayersNum


# summary(list_of_models[23][0], input_size=[1,3,224,224],depth=4)

# for i in range(calDetailModelLayersNum(model_0)):
#     try : 
#         summary(list_of_models[i][0], input_size=[1,3,224,224],depth=4)
#     except:
#         print("mat error : {} layer".format(i))

In [22]:
# @ unit test : model versions eval ability
summary(list_of_models[2][0], input_size=[1,3,224,224])
# print(list_of_models[1][0])
# print(list_of_models[0][0])
# type(list_of_models[0][0])
# type(list_of_models[1][0])

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 512, 7, 7]            --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│

In [23]:
# @ testing : CNN model motification
# from other_program.custom_model.VGG16 import VGG16
# import inspect

# model_0dot0 = VGG16()

# list_of_dummy_model = createDetailLayerVersions(model_0dot0)

# summary(list_of_dummy_model[1][0], [1,3,224,224])

# for i in range(calDetailModelLayersNum(model_0dot0)):
#     try : 
#         summary(list_of_dummy_model[i][0], input_size=[1,3,224,224],depth=4)
#     except:
#         print("mat error : {} layer".format(i))

In [ ]:

test_features , test_labels = dataloaderToFeatureData(model_0, test_dataloader,device)

### 記錄下不同layer的輸出接到xgb的結果

In [48]:
from helperFunction.helperFunctions import flattenTensor
# print(recordXGBoutput(model_0, train_dataloader, test_dataloader, "resnet", {}))

XGB_res = {"model_name":[],
       "output_size":[],
       "num_parm":[],
       "acc":[],
       "f1":[],
       "iters":[],
       "CNN eval time":[],
       "XGB eval time":[],
       }

layer_cnt = 0

for model, model_name in list_of_models:
    input_size = ""
    # 獲取model的output size
    with torch.no_grad():
        try:
            dummy_output = model(torch.rand([1, 3, 224, 224]).to(device))
        except RuntimeError:
            print("mat mismatch!!")
            continue
        except MemoryError:
            print("Not enough memory!! escape form loop")
            break
        except:
            print("error catch ")
            break
        
        input_size = str(dummy_output.shape)[11:-1]

    print("input size is :{}".format(input_size))

    # training XGB !!
    try:
        f1, acc, iter, CNN_eval_time, XGB_eval_time = recordXGBoutput(model, train_dataloader, test_dataloader,enable_muti_module=enable_muti_modal_signal)
    except MemoryError:
        print("Not enough memory!! escape form loop")
        break
    except RuntimeError:
        print("mat mismatch!!")
        continue
    # except:
    #     print("error catch ")
    #     break

    print("best acc is : {}".format(acc[iter]))
    print("best f1 is : {}".format(f1[iter]))
    print("XGB eval time : {}".format(XGB_eval_time) )

    print("model_name : {}".format(model_name) )
    # cal output size
    
    XGB_res["model_name"].append(model_name)
    XGB_res["output_size"].append(input_size)
    XGB_res["num_parm"].append(str(flattenTensor(dummy_output).shape)[11:-1])
    cur_acc = acc[iter]
    XGB_res["acc"].append((-cur_acc) if cur_acc < 0 else cur_acc)
    cur_f1 = f1[iter]
    XGB_res["f1"].append((-cur_f1) if cur_f1 < 0 else cur_f1)
    XGB_res["iters"].append(iter)
    XGB_res["CNN eval time"].append(CNN_eval_time)
    XGB_res["XGB eval time"].append(XGB_eval_time)

    layer_cnt += 1
    if layer_cnt >= 12:
        break



# 將字典轉換為DataFrame，但這次是轉置後的形式
df_transposed = pd.DataFrame.from_dict(XGB_res, orient='index').transpose()

# 將轉置後的DataFrame存儲為CSV檔案
csv_file_path_transposed = "XGB_model_results_" + model_0_name + ".csv"
df_transposed.to_csv(csv_file_path_transposed, index=False)

# 返回CSV檔案的儲存路徑
csv_file_path_transposed

asyncio.run(
    async_discord_bot_notifier("XGB訓練完成!")
)

input size is :[1, 7]
cal CNN model output...
feature size is :2003
feature size is :8012
feature size is : (2003, 48)
<generator object recordXGBoutput.<locals>.<genexpr> at 0x000002D023828AC0>
Indices of NaN values: [(1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (8, 9), (8, 10), (8, 11), (8, 12), (8, 13), (16, 9), (16, 10), (16, 11), (16, 12), (16, 13), (24, 9), (24, 10), (24, 11), (24, 12), (24, 13), (34, 9), (34, 10), (34, 11), (34, 12), (34, 13), (39, 9), (39, 10), (39, 11), (39, 12), (39, 13), (46, 9), (46, 10), (46, 11), (46, 12), (46, 13), (77, 9), (77, 10), (77, 11), (77, 12), (77, 13), (84, 9), (84, 10), (84, 11), (84, 12), (84, 13), (136, 9), (136, 10), (136, 11), (136, 12), (136, 13), (138, 9), (138, 10), (138, 11), (138, 12), (138, 13), (157, 9), (157, 10), (157, 11), (157, 12), (157, 13), (164, 9), (164, 10), (164, 11), (164, 12), (164, 13), (166, 9), (166, 10), (166, 11), (166, 12), (166, 13), (177, 9), (177, 10), (177, 11), (177, 12), (177, 13), (179, 9), (179, 10), (179,

KeyboardInterrupt: 

### 紀錄其他ML模型的結果

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from helperFunction.helperFunctions import flattenTensor
# print(recordXGBoutput(model_0, train_dataloader, test_dataloader, "resnet", {}))

# clf setting 

# csv_model_name , clf = "Linear" , LogisticRegression()
# csv_model_name , clf = "SVC" , SVC(kernel='rbf',gamma='auto', probability=True)
csv_model_name , clf = "RandomForest" , RandomForestClassifier(n_estimators=100)
# csv_model_name , clf = "XGB105" , xgb.XGBClassifier()



linear_res = {"model_name":[],
       "output_size":[],
       "num_parm":[],
       "acc":[],
       "f1":[],
       "eval_time":[]}


layer_cnt = 0

for model, model_name in list_of_models:
    input_size = ""
    with torch.no_grad():
        try:
            dummy_output = model(torch.rand([1, 3, 244, 244]).to(device))
        except:
            print("ERROR!!!")
            break
        input_size = str(dummy_output.shape)[11:-1]

    print("input size is :{}".format(input_size))

    with torch.no_grad():
        # try:
        f1, acc = recordMLoutput(clf, model, train_dataloader, test_dataloader,enable_muti_module=enable_muti_modal_signal)
        # except:
        #     print("ERROR!!!")
        #     break


    # cal output size
    
    linear_res["model_name"].append(model_name)
    linear_res["output_size"].append(input_size)
    linear_res["num_parm"].append(str(flattenTensor(dummy_output).shape)[11:-1])
    
    linear_res["acc"].append(acc)
    linear_res["f1"].append(f1)

    layer_cnt += 1
    if layer_cnt >= 15:
        break

    print(model_name)



# 將字典轉換為DataFrame，但這次是轉置後的形式
df_transposed = pd.DataFrame.from_dict(linear_res, orient='index').transpose()

# 將轉置後的DataFrame存儲為CSV檔案
csv_file_path_transposed = csv_model_name + "_model_results_" +  model_select_signal + ".csv"
df_transposed.to_csv(csv_file_path_transposed, index=False)

# 返回CSV檔案的儲存路徑
csv_file_path_transposed

# asyncio.run(discord_bot_notifier("ML model訓練完成!"))

input size is :[1, 7]
cal CNN model output...
feature size is :2003
<generator object recordMLoutput.<locals>.<genexpr> at 0x000002D02382B3E0>
Indices of NaN values: [(1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (8, 9), (8, 10), (8, 11), (8, 12), (8, 13), (16, 9), (16, 10), (16, 11), (16, 12), (16, 13), (24, 9), (24, 10), (24, 11), (24, 12), (24, 13), (34, 9), (34, 10), (34, 11), (34, 12), (34, 13), (39, 9), (39, 10), (39, 11), (39, 12), (39, 13), (46, 9), (46, 10), (46, 11), (46, 12), (46, 13), (77, 9), (77, 10), (77, 11), (77, 12), (77, 13), (84, 9), (84, 10), (84, 11), (84, 12), (84, 13), (136, 9), (136, 10), (136, 11), (136, 12), (136, 13), (138, 9), (138, 10), (138, 11), (138, 12), (138, 13), (157, 9), (157, 10), (157, 11), (157, 12), (157, 13), (164, 9), (164, 10), (164, 11), (164, 12), (164, 13), (166, 9), (166, 10), (166, 11), (166, 12), (166, 13), (177, 9), (177, 10), (177, 11), (177, 12), (177, 13), (179, 9), (179, 10), (179, 11), (179, 12), (179, 13), (230, 9), (230, 10), (23

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values